In [1]:
import warnings

import csv
import inspect
import pymorphy2

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from allennlp.common.util import START_SYMBOL, END_SYMBOL
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.iterators import BucketIterator
from allennlp.data.tokenizers import Token, CharacterTokenizer
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import AdditiveAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.time_distributed import TimeDistributed
from allennlp.modules.token_embedders import Embedding
from allennlp.nn import util
from allennlp.nn.activations import Activation
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.training.trainer import Trainer

from num2words import num2words
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("/mnt/hdd1/users/svinkapeppa/norm/ru_train.csv")
test = pd.read_csv("/mnt/hdd1/users/svinkapeppa/norm/ru_test_2.csv")

# Part 1. EDA

## Class

Примеры токенов каждого класса

In [3]:
print("{:<12s}{:<25s}\t{}".format("CLASS", "BEFORE", "AFTER"))
print("-" * 45)

for cls in data["class"].unique():
    sample = data[data["class"] == cls].iloc[1]
    
    print("{:<12s}{:<25s}\t{}".format(sample["class"], sample["before"], sample["after"]))

CLASS       BEFORE                   	AFTER
---------------------------------------------
PLAIN       состоянию                	состоянию
DATE        1811 года                	тысяча восемьсот одиннадцатого года
PUNCT       .                        	.
ORDINAL     1895                     	тысяча восемьсот девяносто пятом
VERBATIM    兄                        	兄
LETTERS     РСФСР                    	р с ф с р
CARDINAL    2014                     	две тысячи четырнадцать
MEASURE     31 минуту                	тридцати одной минуту
TELEPHONE   973-96857-7-3            	девятьсот семьдесят три sil девятьсот шестьдесят восемь пятьдесят семь sil семь sil три
ELECTRONIC  About.com                	э_trans б_trans а_trans у_trans т_trans точка к_trans о_trans м_trans
DECIMAL     1 млн                    	одного миллиона
DIGIT       007                      	ноль ноль семь
FRACTION    653/26                   	шестьсот пятьдесят три двадцать шестых
MONEY       2 миллионов долларов     	двух миллио

## Rules

### PLAIN

Токены это класса нормализуются только если форма `before` не является корректным словом на русском языке

Правило: применяем транслитерацию 

In [4]:
data[(data["class"] == "PLAIN") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
5,1,0,PLAIN,Оснащались,Оснащались
6,1,1,PLAIN,латными,латными


In [5]:
data[(data["class"] == "PLAIN") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
43,3,7,PLAIN,Tiberius,т_trans и_trans б_trans е_trans р_trans и_tran...
44,3,8,PLAIN,Julius,д_trans ж_trans у_trans л_trans и_trans у_tran...
45,3,9,PLAIN,Pollienus,п_trans о_trans л_trans л_trans и_trans е_tran...
46,3,10,PLAIN,Auspex,о_trans с_trans п_trans е_trans к_trans с_trans
153,10,4,PLAIN,Half,х_trans а_trans л_trans ф_trans


### DATE

Правило:
1. Цифры из даты преобразуются в слова с соответствующим склонением
2. Простые аббревиатуры расшифровываются в слова с соответсвующим склонением
3. Простые слова остаются как есть

Важно отметить, что иногда нормализаця не применяется в следующих случаях:
1. Дата записана в каком-то странном формате
2. Если присутствуют сложные аббревиатуры
3. По неизвестным причинам иногда не применяется к нормальным датам

In [6]:
data[(data["class"] == "DATE") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
1810,129,7,DATE,130 гг. н. э.,130 гг. н. э.
31178,2291,0,DATE,24 декабря 1913,24 декабря 1913
149594,10857,13,DATE,9/7/004,9/7/004
178971,13002,5,DATE,1 января 2009,1 января 2009
178973,13002,7,DATE,2015 гг.,2015 гг.


In [7]:
data[(data["class"] == "DATE") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
17,2,2,DATE,1811 года,тысяча восемьсот одиннадцатого года
85,6,1,DATE,12 февраля 2013,двенадцатого февраля две тысячи тринадцатого года
90,6,6,DATE,15 февраля 2013,пятнадцатого февраля две тысячи тринадцатого года
189,13,1,DATE,1905 года,тысяча девятьсот пятого года


### PUNCT

Данные токены остаются как есть

In [8]:
data[(data["class"] == "PUNCT") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
4,0,4,PUNCT,.,.
14,1,9,PUNCT,.,.
18,2,3,PUNCT,",",","
24,2,9,PUNCT,(,(
27,2,12,PUNCT,),)


In [9]:
data[(data["class"] == "PUNCT") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after


### ORDINAL

Правило:
1. Удаляем лишнию пунктуацию
2. Цифры из даты преобразуются в слова с соответствующим склонением

Иногда по неизвестным причинам нормализация не применяется

In [10]:
data[(data["class"] == "ORDINAL") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
5469,414,4,ORDINAL,1980,1980
5470,414,5,ORDINAL,-1990-х,-1990-х
55778,4100,25,ORDINAL,80-е,80-е
219291,15893,1,ORDINAL,1948,1948
219292,15893,2,ORDINAL,-1950-х,-1950-х


In [11]:
data[(data["class"] == "ORDINAL") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
53,3,17,ORDINAL,III,третьего
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого
487,35,7,ORDINAL,1802,тысяча восемьсот второй


### VERBATIM

Правило:
1. Греческие буквы переводятся в соответствующие русские слова
2. Буквы с умлаутом переводятся в буквы без умлаута
3. Специльные символы расшифровываются в русские слова

Нормализация не применяется в следующих случаях:
1. Токен является иероглифом
2. Токен является дефисом

In [12]:
data[(data["class"] == "VERBATIM") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
60,4,4,VERBATIM,-,-
109,7,10,VERBATIM,兄,兄
110,7,11,VERBATIM,貴,貴
125,8,9,VERBATIM,-,-
212,16,7,VERBATIM,-,-


In [13]:
data[(data["class"] == "VERBATIM") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
777,55,2,VERBATIM,ї,и
813,57,11,VERBATIM,&,и
1374,96,29,VERBATIM,&,и
2845,199,8,VERBATIM,&,и
5444,411,7,VERBATIM,&,и


### LETTERS

Правило: убирается пунктуация, все буквы приводятся к нижнему регистру, переводятся на русский и записываются через пробел

Если токен является буквой русского языка, то нормализация не производится

In [14]:
data[(data["class"] == "LETTERS") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
155135,11253,7,LETTERS,н,н
1267366,92135,16,LETTERS,к,к
1614082,117172,3,LETTERS,н,н
1925787,139618,8,LETTERS,н,н
2209883,160057,3,LETTERS,н,н


In [15]:
data[(data["class"] == "LETTERS") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
92,6,8,LETTERS,TV,t v
186,12,9,LETTERS,РСФСР,р с ф с р
301,24,1,LETTERS,СПб,с п б
340,26,1,LETTERS,СПб,с п б
346,26,7,LETTERS,В. А.,в а


### CARDINAL

Правило: число записывается словами с сохранением склонения

Иногда, если токен является цифрой, нормализация не производится

In [16]:
data[(data["class"] == "CARDINAL") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
869,61,9,CARDINAL,2,2
9901,738,2,CARDINAL,1,1
40431,2962,18,CARDINAL,3,3
52898,3895,12,CARDINAL,3,3
60271,4417,10,CARDINAL,3,3


In [17]:
data[(data["class"] == "CARDINAL") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
137,9,9,CARDINAL,254,двести пятьдесят четыре
272,21,7,CARDINAL,2014,две тысячи четырнадцать
275,21,10,CARDINAL,12,двенадцать
304,24,4,CARDINAL,2014,две тысячи четырнадцать
343,26,4,CARDINAL,2011,две тысячи одиннадцать


### MEASURE

Правило:
1. Числа переводятся в слова с сохранением склонения
2. Единицы измерения переводятся в русский аналог

Иногда, по неизвестным причинам, нормализация не проводится

In [18]:
data[(data["class"] == "MEASURE") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
871,61,11,MEASURE,3 см,3 см
9177,676,0,MEASURE,2 Р.,2 Р.
34797,2548,3,MEASURE,1952 г,1952 г
36179,2647,11,MEASURE,30 м,30 м
40433,2962,20,MEASURE,4 минуты.,4 минуты.


In [19]:
data[(data["class"] == "MEASURE") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
365,27,12,MEASURE,61st,шестьдесят один стоун
412,30,13,MEASURE,31 минуту,тридцати одной минуту
474,34,12,MEASURE,500 км,пятисот километров
605,44,6,MEASURE,80 см.,восемьдесят сантиметров
793,55,18,MEASURE,480 с.,четыреста восемьдесят секунд


### TELEPHONE

Правило:
1. Числа переводятся в слова с сохранением склонения
2. Разделитель переводится в `sil`

In [20]:
data[(data["class"] == "TELEPHONE") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after


In [21]:
data[(data["class"] == "TELEPHONE") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
576,41,2,TELEPHONE,978-5-86007-700-3,девятьсот семьдесят восемь sil пять sil восемь...
4100,302,15,TELEPHONE,973-96857-7-3,девятьсот семьдесят три sil девятьсот шестьдес...
5530,418,1,TELEPHONE,0-87778-245-8,ноль sil восемьсот семьдесят семь семьдесят во...
5663,426,9,TELEPHONE,0-446-69334-0,ноль sil четыреста сорок шесть sil шестьсот де...
5887,443,10,TELEPHONE,5-9 221-0 440-3,пять sil девять sil двести двадцать один sil н...


### ELECTRONIC

Правило:
1. Латинские буквы записываются с помощью транслитерации
2. Числа и цифры переводятся в слова с сохранением склонения
3. Пунктуация расшифровывается в слова

Нормализация не применяется к специальным токенам `::`

In [22]:
data[(data["class"] == "ELECTRONIC") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
31310,2299,2,ELECTRONIC,::,::
129042,9369,8,ELECTRONIC,::,::
314279,22850,5,ELECTRONIC,::,::
314283,22850,9,ELECTRONIC,::,::
401471,29128,5,ELECTRONIC,::,::


In [23]:
data[(data["class"] == "ELECTRONIC") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
1257,89,7,ELECTRONIC,ElvisPresleyBiography.com,э_trans л_trans в_trans и_trans с_trans п_tran...
3157,224,23,ELECTRONIC,About.com,э_trans б_trans а_trans у_trans т_trans точка ...
3883,284,0,ELECTRONIC,Billboard.com,б_trans и_trans л_trans б_trans о_trans р_tran...
5808,437,0,ELECTRONIC,THG.ru,t h g точка р_trans у_trans
9400,695,1,ELECTRONIC,2014-09-01.Charts.org.nz,две тысячи четырнадцать дефис ноль девять дефи...


### DECIMAL

Правило:
1. Числа и цифры переводятся в слова с сохранением склонения
2. Дробная часть вводится через `и`
3. Аббревиатуры и единицы измерения расшифровываются в русские слова

Иногда, по неизвестным причинам, нормализация не производится

In [24]:
data[(data["class"] == "DECIMAL") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
36237,2652,4,DECIMAL,"31,2","31,2"
369113,26770,6,DECIMAL,"-15,16","-15,16"
470663,34143,4,DECIMAL,-40 тыс.,-40 тыс.
471031,34167,8,DECIMAL,"58,27","58,27"
585219,42434,16,DECIMAL,"-73,83","-73,83"


In [25]:
data[(data["class"] == "DECIMAL") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
1292,91,10,DECIMAL,60 тысяч,шестьдесят тысяч
7237,542,7,DECIMAL,1 млн,одного миллиона
7685,575,16,DECIMAL,"1,2",одна целая и две десятых
7687,575,18,DECIMAL,"3,5",три целых и пять десятых
8712,640,16,DECIMAL,"7203,54",семь тысяч двести три целых и пятьдесят четыре...


### DIGIT

Правило: каждая цифра переводится в слово, стоящее в начальной форме

In [26]:
data[(data["class"] == "DIGIT") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after


In [27]:
data[(data["class"] == "DIGIT") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
2524,178,4,DIGIT,06,ноль шесть
9254,683,7,DIGIT,007,ноль ноль семь
22460,1667,4,DIGIT,00,ноль ноль
23209,1722,8,DIGIT,0171,ноль один семь один
36632,2680,22,DIGIT,09,ноль девять


### FRACTION

Правило:
1. Цифры и числа переводятся в слова с сохранением склонения
2. Знаменатель дополнительно склоняется

Иногда, по неизвестным причинам, нормализация не производится

In [28]:
data[(data["class"] == "FRACTION") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
115851,8416,6,FRACTION,-717/714,-717/714
707955,51358,12,FRACTION,-515/9,-515/9
784970,56938,9,FRACTION,-2/9,-2/9
803454,58290,1,FRACTION,-740/741,-740/741
1047578,76111,10,FRACTION,1451525206/9781451525205,1451525206/9781451525205


In [29]:
data[(data["class"] == "FRACTION") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
3578,259,17,FRACTION,1883/1884,тысяча восемьсот восемьдесят три тысяча восемь...
15218,1121,7,FRACTION,653/26,шестьсот пятьдесят три двадцать шестых
23441,1735,11,FRACTION,3/16,трех шестнадцатых
24061,1781,5,FRACTION,2/3,две третьих
28159,2075,17,FRACTION,2014/15,две тысячи четырнадцать пятнадцатых


### MONEY

Правило:
1. Числа и цифры записываются словами с сохранением склонения
2. Аббревиатуры расшифровываются

Нормализация не производится:
1. Если это не похоже на какое-то количество денег (например, `PHP 5`)
2. Иногда, по неизвестным причинам, не производится для нормальной записи какого-то количества денег

In [30]:
data[(data["class"] == "MONEY") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
100783,7367,6,MONEY,€5,€5
112382,8172,5,MONEY,CYP21,CYP21
195934,14199,3,MONEY,"Rs 3,141","Rs 3,141"
242609,17582,17,MONEY,-2 млрд долларов США,-2 млрд долларов США
483531,35077,16,MONEY,CYP2,CYP2


In [31]:
data[(data["class"] == "MONEY") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
12028,883,5,MONEY,194 млн долл.,ста девяноста четырех миллионов долларов
18226,1345,10,MONEY,2 миллионов долларов,двух миллионов долларов
20298,1504,9,MONEY,10 000 рублей,десять тысяч рублей
22832,1696,9,MONEY,200 миллионов рублей,двести миллионов рублей
24820,1835,4,MONEY,250 рублей,двести пятьдесят рублей


### TIME

Правило:
1. Производится, если время записано в формате `HH:MM`
2. Числа и цифры переводятся в слова с сохранением склонения
3. Часовые пояса расшифровываются

Нормализация не производится, если время записано в формате, отличающемся от `HH:MM`

In [32]:
data[(data["class"] == "TIME") & (data["before"] == data["after"])].head()

,sentence_id,token_id,class,before,after
14366,1059,1,TIME,11:19:52,11:19:52
14615,1076,1,TIME,11:20:41,11:20:41
45775,3347,2,TIME,15:03:28,15:03:28
79398,5823,2,TIME,07:33:17,07:33:17
112859,8205,5,TIME,06:53:05,06:53:05


In [33]:
data[(data["class"] == "TIME") & (data["before"] != data["after"])].head()

,sentence_id,token_id,class,before,after
36235,2652,2,TIME,06:06,шесть часов шесть минут
57147,4195,10,TIME,02:33,два часа тридцать три минуты
76065,5575,9,TIME,07:00,семь часов
82739,6064,6,TIME,3:08,три часа восемь минут
82749,6064,16,TIME,3:57,три часа пятьдесят семь минут


## Custom ORDINAL normalization rule

Будем нормализовать только латинские и греческие числа

Если это греческое число:
1. Оставим в нем только греческие цифры
2. Переведем получившееся число в латинское
3. С помощью `num2words` получим порядковую форму

Если это латинское число:
1. Оставим в нем только цифры
2. С помощью `num2words` получим порядковую форму
3. Если у числа изначальна была форма, то попробуем подобрать ее с помощью `pymorphy2`

In [34]:
def roman_to_int(word):
    roman = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
    value = 0

    for i in range(len(word)):
        if i > 0 and roman[word[i]] > roman[word[i - 1]]:
            value += roman[word[i]] - 2 * roman[word[i - 1]]
        else:
            value += roman[word[i]]

    return value


def normalize_roman(word):
    chars = []
    for char in word:
        if char in {"I", "V", "X", "L", "C", "D", "M"}:
            chars += char

    return num2words(roman_to_int("".join(chars)), ordinal=True, lang="ru")


def normalize_latin(word):
    chars = []
    for char in word:
        if char in {"1", "2", "3", "4", "5", "6", "7", "8", "9", "0"}:
            chars += char

    values = num2words("".join(chars), ordinal=True, lang="ru").split()

    form = word[word.find("-") + 1:]
    morph = pymorphy2.MorphAnalyzer()
    for lexeme in morph.parse(values[-1])[0].lexeme:
        lex = lexeme[0]

        if lex.find(form) != -1:
            values = values[:-1] + [lex]
            break

    if values[0] == " одна":
        values = values[1:]

    return " ".join(values)


def normalize(word):
    for char in word:
        if char in {"1", "2", "3", "4", "5", "6", "7", "8", "9", "0"}:
            return normalize_latin(word)
        if char in {"I", "V", "X", "L", "C", "D", "M"}:
            return normalize_roman(word)

    return word

In [35]:
tmp = data[(data["class"] == "ORDINAL") & (data["before"] != data["after"])][:25]
tmp["custom"] = tmp["before"].apply(normalize)
tmp

,sentence_id,token_id,class,before,after,custom
53,3,17,ORDINAL,III,третьего,третий
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом,одна тысяча восемьсот девяносто пятый
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом,одна тысяча восемьсот девяносто шестой
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого,одна тысяча семьсот девяносто шестой
487,35,7,ORDINAL,1802,тысяча восемьсот второй,одна тысяча восемьсот второй
540,38,5,ORDINAL,1109,тысяча сто девятого,одна тысяча сто девятый
541,38,6,ORDINAL,—1112,тысяча сто двенадцатого,одна тысяча сто двенадцатый
940,67,3,ORDINAL,1-й,первый,одной
1764,126,3,ORDINAL,22-х,двадцать вторых,двадцать двух
2195,155,3,ORDINAL,1950-х,тысяча девятьсот пятидесятых,одна тысяча девятьсот пятидесятых


# Part 2. Seq2Seq

Создадим обучающую и валидационную выборку:
1. Чтобы не ждать очень долго, возьмем только 10% токенов каждого класса
2. Обучаться будем на 80%, валидироваться на 20%

In [3]:
indices = test['sentence_id'].astype(str) + '_' + test['token_id'].astype(str)

In [4]:
train, valid = [], []
for cls in data["class"].unique():
    tmp = shuffle(data[data["class"] == cls])
    tmp = tmp[:int(tmp.shape[0] * 0.1)]
    
    train += [tmp[:int(tmp.shape[0] * 0.8)]]
    valid += [tmp[int(tmp.shape[0] * 0.8):]]

train = shuffle(pd.concat(train))
valid = shuffle(pd.concat(valid))

train = train.drop(["sentence_id", "token_id", "class"], axis=1)
valid = valid.drop(["sentence_id", "token_id", "class"], axis=1)
test = test.drop(["sentence_id", "token_id"], axis=1)

train.to_csv("/mnt/hdd1/users/svinkapeppa/norm/train.csv", index=False, header=False)
valid.to_csv("/mnt/hdd1/users/svinkapeppa/norm/valid.csv", index=False, header=False)
test.to_csv("/mnt/hdd1/users/svinkapeppa/norm/test.csv", index=False, header=False)

In [5]:
class Seq2SeqDatasetReader(DatasetReader):
    def __init__(self):
        super().__init__(lazy=False)

        self.source_tokenizer = CharacterTokenizer()
        self.target_tokenizer = CharacterTokenizer()
        self.source_token_indexers = {"tokens": SingleIdTokenIndexer(namespace="tokens")}
        self.target_token_indexers = {"tokens": SingleIdTokenIndexer(namespace="target_tokens")}

    def text_to_instance(self, source, target):
        source_tokens = self.source_tokenizer.tokenize(source)
        source_tokens.insert(0, Token(START_SYMBOL))
        source_tokens.append(Token(END_SYMBOL))
        source_field = TextField(source_tokens, self.source_token_indexers)

        if target:
            target_tokens = self.target_tokenizer.tokenize(target)
            target_tokens.insert(0, Token(START_SYMBOL))
            target_tokens.append(Token(END_SYMBOL))
            target_field = TextField(target_tokens, self.target_token_indexers)

            return Instance({"source_tokens": source_field, "target_tokens": target_field})

        return Instance({"source_tokens": source_field})

    def _read(self, file_path):
        with open(file_path) as f:
            for row in csv.reader(f):
                try:
                    source, target = row
                except ValueError:
                    source, target = row[0], None

                yield self.text_to_instance(source, target)

In [6]:
reader = Seq2SeqDatasetReader()

In [7]:
train_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/train.csv")
valid_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/valid.csv")
test_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/test.csv")

845950it [00:43, 19543.61it/s]
211495it [00:10, 19669.16it/s]
989880it [00:30, 32970.82it/s]


In [8]:
vocab = Vocabulary.from_instances(train_dataset + valid_dataset)

100%|██████████| 1057445/1057445 [00:15<00:00, 68846.07it/s]


In [2]:
def to_text(tokens):
    return "".join(map(str, list(tokens[1:-1])))


def predict(model, dataset):
    predictions = []

    for i in tqdm(range(0, len(dataset), 128)):
        batch = dataset[i: i + 128]
        tmp = ["".join(prediction["predicted_tokens"]) for prediction in model.forward_on_instances(batch)]
        predictions += tmp

    return predictions


def sample(model, dataset):
    predictions = predict(model, dataset)
    
    for idx, instance in enumerate(dataset):
        print("BEFORE: {}".format("".join(to_text(instance.fields["source_tokens"].tokens))))
        print("AFTER:  {}".format(predictions[idx]))
        print("TARGET: {}".format("".join(to_text(instance.fields["target_tokens"].tokens))))
        print()

### DotProductAttention

In [10]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

source_embedder = BasicTextFieldEmbedder(
    {"tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM)}
)
encoder = PytorchSeq2SeqWrapper(nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
attention = DotProductAttention()

model = SimpleSeq2Seq(
    vocab=vocab,
    source_embedder=source_embedder,
    encoder=encoder,
    max_decoding_steps=256,
    attention=attention,
    beam_size=8,
    target_namespace="target_tokens",
    target_embedding_dim=HIDDEN_DIM,
    use_bleu=True,
)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    num_epochs=2,
    patience=10,
    cuda_device=0,
)

trainer.train()

loss: 0.2423 ||: 100%|██████████| 26436/26436 [45:18<00:00,  9.72it/s] 
BLEU: 0.9331, loss: 0.0229 ||: 100%|██████████| 6610/6610 [39:52<00:00,  2.76it/s]
loss: 0.0193 ||: 100%|██████████| 26436/26436 [44:57<00:00,  9.80it/s]  
BLEU: 0.9534, loss: 0.0181 ||: 100%|██████████| 6610/6610 [38:45<00:00,  2.84it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 12034.032,
 'peak_gpu_0_memory_MB': 1530,
 'peak_gpu_1_memory_MB': 188,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '2:48:55.026740',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_loss': 0.019266804745127816,
 'training_cpu_memory_MB': 12034.032,
 'training_gpu_0_memory_MB': 1530,
 'training_gpu_1_memory_MB': 188,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_BLEU': 0.9534079176819831,
 'validation_loss': 0.018127608652418907,
 'best_validation_BLEU': 0.9534079176819831,
 'best_validation_loss': 0.018127608652418907}

In [11]:
sample(model, valid_dataset[:5])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.



BEFORE: Земле
AFTER:  Земле
TARGET: Земле

BEFORE: Имст
AFTER:  Имст
TARGET: и м с т

BEFORE: (
AFTER:  (
TARGET: (

BEFORE: попугая
AFTER:  попугая
TARGET: попугая

BEFORE: латные
AFTER:  латные
TARGET: латные



In [12]:
df = pd.DataFrame(zip(indices, predict(model, test_dataset)), columns=["id", "after"])
df.to_csv(
    "/mnt/hdd1/users/svinkapeppa/norm/dot_product_attention.csv",
    index=False,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC
)

### AdditiveAttention

In [10]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

source_embedder = BasicTextFieldEmbedder(
    {"tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM)}
)
encoder = PytorchSeq2SeqWrapper(nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
attention = AdditiveAttention(HIDDEN_DIM, HIDDEN_DIM)

model = SimpleSeq2Seq(
    vocab=vocab,
    source_embedder=source_embedder,
    encoder=encoder,
    max_decoding_steps=256,
    attention=attention,
    beam_size=8,
    target_namespace="target_tokens",
    target_embedding_dim=HIDDEN_DIM,
    use_bleu=True,
)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    num_epochs=2,
    patience=10,
    cuda_device=0,
)

trainer.train()

loss: 0.2022 ||: 100%|██████████| 26436/26436 [55:18<00:00,  7.97it/s]  
BLEU: 0.9490, loss: 0.0195 ||: 100%|██████████| 6610/6610 [44:34<00:00,  2.47it/s]
loss: 0.0186 ||: 100%|██████████| 26436/26436 [55:10<00:00,  7.99it/s]  
BLEU: 0.9553, loss: 0.0173 ||: 100%|██████████| 6610/6610 [50:17<00:00,  2.19it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 12032.424,
 'peak_gpu_0_memory_MB': 5876,
 'peak_gpu_1_memory_MB': 187,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '3:25:21.574639',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_loss': 0.018573970868112383,
 'training_cpu_memory_MB': 12032.424,
 'training_gpu_0_memory_MB': 5876,
 'training_gpu_1_memory_MB': 187,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_BLEU': 0.9553282927246081,
 'validation_loss': 0.01733670553757225,
 'best_validation_BLEU': 0.9553282927246081,
 'best_validation_loss': 0.01733670553757225}

In [11]:
sample(model, valid_dataset[:5])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.



BEFORE: .
AFTER:  .
TARGET: .

BEFORE: судьбы
AFTER:  судьбы
TARGET: судьбы

BEFORE: Джон
AFTER:  Джон
TARGET: Джон

BEFORE: и
AFTER:  и
TARGET: и

BEFORE: евразийства
AFTER:  евразийства
TARGET: евразийства



In [12]:
df = pd.DataFrame(zip(indices, predict(model, test_dataset)), columns=["id", "after"])
df.to_csv(
    "/mnt/hdd1/users/svinkapeppa/norm/additive_attention.csv",
    index=False,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC
)

### BilinearAttention

In [10]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

source_embedder = BasicTextFieldEmbedder(
    {"tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM)}
)
encoder = PytorchSeq2SeqWrapper(nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)

model = SimpleSeq2Seq(
    vocab=vocab,
    source_embedder=source_embedder,
    encoder=encoder,
    max_decoding_steps=256,
    attention=attention,
    beam_size=8,
    target_namespace="target_tokens",
    target_embedding_dim=HIDDEN_DIM,
    use_bleu=True,
)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    num_epochs=2,
    patience=10,
    cuda_device=0,
)

trainer.train()

loss: 0.2235 ||: 100%|██████████| 26436/26436 [51:52<00:00,  8.49it/s]  
BLEU: 0.8747, loss: 0.0825 ||: 100%|██████████| 6610/6610 [54:50<00:00,  2.01it/s]  
loss: 0.0329 ||: 100%|██████████| 26436/26436 [52:19<00:00,  8.42it/s]  
BLEU: 0.9330, loss: 0.0306 ||: 100%|██████████| 6610/6610 [41:59<00:00,  2.62it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 12035.244,
 'peak_gpu_0_memory_MB': 1638,
 'peak_gpu_1_memory_MB': 187,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '3:21:02.820026',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_loss': 0.03286595143203915,
 'training_cpu_memory_MB': 12035.244,
 'training_gpu_0_memory_MB': 1638,
 'training_gpu_1_memory_MB': 187,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_BLEU': 0.9329769361271228,
 'validation_loss': 0.030575247813249006,
 'best_validation_BLEU': 0.9329769361271228,
 'best_validation_loss': 0.030575247813249006}

In [11]:
sample(model, valid_dataset[:5])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.



BEFORE: герба
AFTER:  герба
TARGET: герба

BEFORE: прочная
AFTER:  прочная
TARGET: прочная

BEFORE: Она
AFTER:  Она
TARGET: Она

BEFORE: Чадайкин
AFTER:  Чадайкин
TARGET: Чадайкин

BEFORE: ее
AFTER:  ее
TARGET: ее



In [12]:
df = pd.DataFrame(zip(indices, predict(model, test_dataset)), columns=["id", "after"])
df.to_csv(
    "/mnt/hdd1/users/svinkapeppa/norm/bilinear_attention.csv",
    index=False,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC
)

### Transformer

In [10]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

source_embedder = BasicTextFieldEmbedder(
    {"tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM)}
)
encoder = StackedSelfAttentionEncoder(
    input_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    projection_dim=HIDDEN_DIM,
    feedforward_hidden_dim=HIDDEN_DIM,
    num_layers=4,
    num_attention_heads=8,
)
attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)

model = SimpleSeq2Seq(
    vocab=vocab,
    source_embedder=source_embedder,
    encoder=encoder,
    max_decoding_steps=256,
    attention=attention,
    beam_size=8,
    target_namespace="target_tokens",
    target_embedding_dim=HIDDEN_DIM,
    use_bleu=True,
)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    num_epochs=2,
    patience=10,
    cuda_device=0,
)

trainer.train()

loss: 1.7061 ||: 100%|██████████| 26436/26436 [57:43<00:00,  7.63it/s]  
BLEU: 0.0005, loss: 2.4592 ||: 100%|██████████| 6610/6610 [18:44<00:00,  5.88it/s]
loss: 1.4617 ||: 100%|██████████| 26436/26436 [56:58<00:00,  7.73it/s] 
BLEU: 0.0021, loss: 2.1365 ||: 100%|██████████| 6610/6610 [19:58<00:00,  5.52it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 12035.536,
 'peak_gpu_0_memory_MB': 1768,
 'peak_gpu_1_memory_MB': 187,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '2:33:25.533018',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_loss': 1.4617354117546957,
 'training_cpu_memory_MB': 12035.536,
 'training_gpu_0_memory_MB': 1768,
 'training_gpu_1_memory_MB': 187,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_BLEU': 0.0021054491277892624,
 'validation_loss': 2.136469397449277,
 'best_validation_BLEU': 0.0021054491277892624,
 'best_validation_loss': 2.136469397449277}

In [11]:
sample(model, valid_dataset[:5])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.



BEFORE: наконечника
AFTER:  на
TARGET: наконечника

BEFORE: им
AFTER:  под
TARGET: им

BEFORE: англ
AFTER:  что
TARGET: англ

BEFORE: особой
AFTER:  что
TARGET: особой

BEFORE: также
AFTER:  что
TARGET: также



In [12]:
df = pd.DataFrame(zip(indices, predict(model, test_dataset)), columns=["id", "after"])
df.to_csv(
    "/mnt/hdd1/users/svinkapeppa/norm/transformer.csv",
    index=False,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC
)

# Part 3. Features

## Text Classification

In [12]:
data = pd.read_csv("/mnt/hdd1/users/svinkapeppa/norm/ru_train.csv")
test = pd.read_csv("/mnt/hdd1/users/svinkapeppa/norm/ru_test_2.csv")

In [13]:
indices = test['sentence_id'].astype(str) + '_' + test['token_id'].astype(str)

In [4]:
train, valid = [], []
for cls in data["class"].unique():
    tmp = shuffle(data[data["class"] == cls])
    tmp = tmp[:int(tmp.shape[0] * 0.1)]
    
    train += [tmp[:int(tmp.shape[0] * 0.8)]]
    valid += [tmp[int(tmp.shape[0] * 0.8):]]

train = shuffle(pd.concat(train))
valid = shuffle(pd.concat(valid))

train = train.drop(["sentence_id", "token_id"], axis=1)
valid = valid.drop(["sentence_id", "token_id"], axis=1)
test = test.drop(["sentence_id", "token_id"], axis=1)

train.to_csv("/mnt/hdd1/users/svinkapeppa/norm/train.csv", index=False, header=False)
valid.to_csv("/mnt/hdd1/users/svinkapeppa/norm/valid.csv", index=False, header=False)
test.to_csv("/mnt/hdd1/users/svinkapeppa/norm/test.csv", index=False, header=False)

In [5]:
class ClassDatasetReader(DatasetReader):
    def __init__(self):
        super().__init__(lazy=False)

        self.tokenizer = CharacterTokenizer()
        self.token_indexers = {"tokens": SingleIdTokenIndexer()}
        self.class_indexers = {"tokens": SingleIdTokenIndexer(namespace="class")}

    def text_to_instance(self, token, cls=None):
        tokens = self.tokenizer.tokenize(token)
        tokens_field = TextField(tokens, self.token_indexers)

        if cls:
            class_field = TextField([Token(cls)], self.class_indexers)
            return Instance({"tokens": tokens_field, "cls": class_field})

        return Instance({"tokens": tokens_field})

    def _read(self, file_path):
        with open(file_path) as f:
            for row in csv.reader(f):
                try:
                    cls, token, _ = row
                except ValueError:
                    cls, token, _ = None, row[0], None

                yield self.text_to_instance(token, cls)

In [6]:
reader = ClassDatasetReader()
train_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/train.csv")
valid_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/valid.csv")
test_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/test.csv")

845950it [00:25, 33022.11it/s]
211495it [00:06, 33425.58it/s]
989880it [00:22, 44006.43it/s]


In [7]:
vocab = Vocabulary.from_instances(train_dataset + valid_dataset)

100%|██████████| 1057445/1057445 [00:07<00:00, 150357.07it/s]


In [8]:
class LstmTagger(Model):
    def __init__(self, embedder, encoder, vocab):
        super().__init__(vocab)

        self.embedder = embedder
        self.encoder = encoder
        self.out = nn.Linear(encoder.get_output_dim(), vocab.get_vocab_size("class"))

        self.accuracy = CategoricalAccuracy()

    def forward(self, tokens, cls=None):
        mask = get_text_field_mask(tokens)

        embeddings = self.embedder(tokens)
        encodings = torch.mean(self.encoder(embeddings, mask), dim=1)
        logits = self.out(encodings)

        output = {"tag_logits": logits}

        if cls is not None:
            self.accuracy(logits, cls["tokens"][:, 0], mask[:, 0])
            output["loss"] = sequence_cross_entropy_with_logits(logits, cls["tokens"], mask[:, 0])

        return output

    def get_metrics(self, reset=False):
        return {"accuracy": self.accuracy.get_metric(reset)}

In [9]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

embedder = BasicTextFieldEmbedder({
    "tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM),
})
encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = LstmTagger(embedder, encoder, vocab)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    patience=10,
    num_epochs=2,
    cuda_device=0,
)

trainer.train()

accuracy: 0.9929, loss: 0.8721 ||: 100%|██████████| 26436/26436 [05:32<00:00, 79.42it/s]  
accuracy: 0.9960, loss: 0.4426 ||: 100%|██████████| 6610/6610 [00:47<00:00, 139.61it/s]
accuracy: 0.9960, loss: 0.4348 ||: 100%|██████████| 26436/26436 [05:17<00:00, 83.22it/s] 
accuracy: 0.9963, loss: 0.4208 ||: 100%|██████████| 6610/6610 [00:46<00:00, 143.47it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 9631.368,
 'peak_gpu_0_memory_MB': 644,
 'peak_gpu_1_memory_MB': 188,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '0:12:24.726766',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_accuracy': 0.9960494072337694,
 'training_loss': 0.434794259539872,
 'training_cpu_memory_MB': 9631.368,
 'training_gpu_0_memory_MB': 644,
 'training_gpu_1_memory_MB': 187,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_accuracy': 0.9963497955034398,
 'validation_loss': 0.420812424877588,
 'best_validation_accuracy': 0.9963497955034398,
 'best_validation_loss': 0.420812424877588}

In [10]:
predictions = []

for i in tqdm(range(0, len(test_dataset), 128)):
    batch = test_dataset[i: i + 128]
    tmp = [vocab.get_index_to_token_vocabulary("class")[np.argmax(prediction["tag_logits"])]
           for prediction in model.forward_on_instances(batch)]
    predictions += tmp

In [17]:
test["class"] = predictions
test = test[["class", "before"]]

In [19]:
train.to_csv("/mnt/hdd1/users/svinkapeppa/norm/train_class.csv", index=False, header=False)
valid.to_csv("/mnt/hdd1/users/svinkapeppa/norm/valid_class.csv", index=False, header=False)
test.to_csv("/mnt/hdd1/users/svinkapeppa/norm/test_class.csv", index=False, header=False)

## Seq2Seq Text Normalization

In [3]:
class Seq2SeqDatasetReader(DatasetReader):
    def __init__(self):
        super().__init__(lazy=False)

        self.source_tokenizer = CharacterTokenizer()
        self.target_tokenizer = CharacterTokenizer()
        self.source_token_indexers = {"tokens": SingleIdTokenIndexer(namespace="tokens")}
        self.class_token_indexers = {"tokens": SingleIdTokenIndexer(namespace="class")}
        self.target_token_indexers = {"tokens": SingleIdTokenIndexer(namespace="target_tokens")}

    def text_to_instance(self, source, cls, target):
        source_tokens = self.source_tokenizer.tokenize(source)
        source_tokens.insert(0, Token(START_SYMBOL))
        source_tokens.append(Token(END_SYMBOL))
        source_field = TextField(source_tokens, self.source_token_indexers)

        cls = [Token(cls) for _ in range(len(source))]
        cls.insert(0, Token(START_SYMBOL))
        cls.append(Token(END_SYMBOL))
        class_field = TextField(cls, self.class_token_indexers)

        if target:
            target_tokens = self.target_tokenizer.tokenize(target)
            target_tokens.insert(0, Token(START_SYMBOL))
            target_tokens.append(Token(END_SYMBOL))
            target_field = TextField(target_tokens, self.target_token_indexers)

            return Instance({
                "source_tokens": source_field,
                "class_tokens": class_field,
                "target_tokens": target_field
            })

        return Instance({"source_tokens": source_field, "class_tokens": class_field})

    def _read(self, file_path):
        with open(file_path) as f:
            for row in csv.reader(f):
                try:
                    cls, source, target = row
                except ValueError:
                    (cls, source), target = row, None

                yield self.text_to_instance(source, cls, target)

In [4]:
reader = Seq2SeqDatasetReader()

In [5]:
train_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/train_class.csv")
valid_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/valid_class.csv")
test_dataset = reader.read("/mnt/hdd1/users/svinkapeppa/norm/test_class.csv")

845950it [01:01, 13730.77it/s]
211495it [00:15, 13885.91it/s]
989880it [00:47, 20921.00it/s]


In [6]:
vocab = Vocabulary.from_instances(train_dataset + valid_dataset)

100%|██████████| 1057445/1057445 [00:21<00:00, 49895.38it/s]


In [7]:
class Model(SimpleSeq2Seq):
    def __init__(
        self,
        vocab=None,
        source_embedder=None,
        encoder=None,
        max_decoding_steps=None,
        attention=None,
        attention_function=None,
        beam_size=None,
        target_namespace="tokens",
        target_embedding_dim=None,
        scheduled_sampling_ratio=0.,
        use_bleu=True,
    ):
        super().__init__(
            vocab,
            source_embedder,
            encoder,
            max_decoding_steps,
            attention,
            attention_function,
            beam_size,
            target_namespace,
            target_embedding_dim,
            scheduled_sampling_ratio,
            use_bleu,
        )

    def _encode(self, source_tokens, class_tokens):
        embedded_input = self._source_embedder({
            "tokens": source_tokens,
            "class": class_tokens,
        })
        source_mask = util.get_text_field_mask(source_tokens)
        encoder_outputs = self._encoder(embedded_input, source_mask)

        return {
            "source_mask": source_mask,
            "encoder_outputs": encoder_outputs,
        }

    def forward(self, source_tokens, class_tokens, target_tokens=None):
        state = self._encode(source_tokens, class_tokens)

        if target_tokens:
            state = self._init_decoder_state(state)
            output_dict = self._forward_loop(state, target_tokens)
        else:
            output_dict = {}

        if not self.training:
            state = self._init_decoder_state(state)
            predictions = self._forward_beam_search(state)
            output_dict.update(predictions)
            if target_tokens and self._bleu:
                top_k_predictions = output_dict["predictions"]
                best_predictions = top_k_predictions[:, 0, :]
                self._bleu(best_predictions, target_tokens["tokens"])

        return output_dict

In [8]:
class CustomBasicTextFieldEmbedder(BasicTextFieldEmbedder):
    def __init__(
        self,
        token_embedders,
        embedder_to_indexer_map=None,
        allow_unmatched_keys=False
    ):
        super().__init__(token_embedders, embedder_to_indexer_map, allow_unmatched_keys)

    def forward(
        self,
        text_field_input,
        num_wrapping_dims=0,
        **kwargs
    ):
        embedder_keys = self._token_embedders.keys()
        input_keys = text_field_input.keys()

        embedded_representations = []
        keys = sorted(embedder_keys)
        for key in keys:
            embedder = getattr(self, 'token_embedder_{}'.format(key))

            forward_params = inspect.signature(embedder.forward).parameters
            forward_params_values = {}
            for param in forward_params.keys():
                if param in kwargs:
                    forward_params_values[param] = kwargs[param]

            for _ in range(num_wrapping_dims):
                embedder = TimeDistributed(embedder)

            tensors = [text_field_input[key]["tokens"]]
            token_vectors = embedder(*tensors, **forward_params_values)
            embedded_representations.append(token_vectors)

        return torch.cat(embedded_representations, dim=-1)

In [9]:
EMBEDDING_DIM = 256
HIDDEN_DIM = 256

source_embedder = CustomBasicTextFieldEmbedder({
    "tokens": Embedding(num_embeddings=vocab.get_vocab_size("tokens"), embedding_dim=EMBEDDING_DIM),
    "class": Embedding(num_embeddings=vocab.get_vocab_size("class"), embedding_dim=EMBEDDING_DIM)
})
encoder = PytorchSeq2SeqWrapper(nn.LSTM(2 * EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
attention = AdditiveAttention(HIDDEN_DIM, HIDDEN_DIM)

model = Model(
    vocab=vocab,
    source_embedder=source_embedder,
    encoder=encoder,
    max_decoding_steps=256,
    attention=attention,
    beam_size=8,
    target_namespace="target_tokens",
    target_embedding_dim=HIDDEN_DIM,
    use_bleu=True,
)
model.to("cuda")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=valid_dataset,
    num_epochs=2,
    patience=10,
    cuda_device=0,
)

trainer.train()

loss: 0.1636 ||: 100%|██████████| 26436/26436 [55:33<00:00,  7.93it/s]  
BLEU: 0.9518, loss: 0.0163 ||: 100%|██████████| 6610/6610 [43:44<00:00,  2.52it/s]
loss: 0.0148 ||: 100%|██████████| 26436/26436 [55:01<00:00,  8.01it/s]  
BLEU: 0.9604, loss: 0.0133 ||: 100%|██████████| 6610/6610 [40:39<00:00,  2.71it/s]


{'best_epoch': 1,
 'peak_cpu_memory_MB': 14416.504,
 'peak_gpu_0_memory_MB': 5134,
 'peak_gpu_1_memory_MB': 188,
 'peak_gpu_2_memory_MB': 19,
 'peak_gpu_3_memory_MB': 19,
 'training_duration': '3:15:00.259118',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_loss': 0.014830387253239822,
 'training_cpu_memory_MB': 14416.504,
 'training_gpu_0_memory_MB': 5134,
 'training_gpu_1_memory_MB': 188,
 'training_gpu_2_memory_MB': 19,
 'training_gpu_3_memory_MB': 19,
 'validation_BLEU': 0.9604490343928831,
 'validation_loss': 0.01328044712052597,
 'best_validation_BLEU': 0.9604490343928831,
 'best_validation_loss': 0.01328044712052597}

In [10]:
sample(model, valid_dataset[:5])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.



BEFORE: типы
AFTER:  типы
TARGET: типы

BEFORE: 133
AFTER:  сто тридцать три
TARGET: сто тридцать три

BEFORE: этого
AFTER:  этого
TARGET: этого

BEFORE: Африки
AFTER:  Африки
TARGET: Африки

BEFORE: настоящее
AFTER:  настоящее
TARGET: настоящее



In [14]:
df = pd.DataFrame(zip(indices, predict(model, test_dataset)), columns=["id", "after"])
df.to_csv(
    "/mnt/hdd1/users/svinkapeppa/norm/additive_attention_class.csv",
    index=False,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC
)

# Part 4. Report